# Run analysis on the data

In [1]:
%run -i ~/Documents/tomology/tomography/utils.py

/home/xf28id2/conda_envs/khalifah_20210725/lib/python3.9/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [2]:
%matplotlib qt

In [3]:
import pyFAI
import pathlib

from ipywidgets import interact_manual

## Create the calculator

In [4]:
C = Calculator()

## Load the Run Data

In [5]:
from databroker import catalog

db = catalog["xpdd"]

In [6]:
UID = '46e695e0-1638-4045-a1ff-1506854d3c13'
run = db[UID]
C.metadata = dict(run.metadata["start"])

In [ ]:
C.frames_arr = run.primary.to_dask()["dexela_image"]
C.frames_arr

## Load the calibration data

In [7]:
PONI_FILE = "/nsls2/data/xpd/legacy/processed/from_mehmet/CeO2_0.25x0.25_beam.poni"
C.ai = pyFAI.load(PONI_FILE) if PONI_FILE else pyFAI.azimuthalIntegrator.AzimuthalIntegrator(dist=200, wavelength=0.186, detector="perkin")

## Choose the Max image

In [8]:
interact_manual(
    show_npy_array, 
    template="/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/PARADIM-1_46e695e0-1638-4045-a1ff-1506854d3c13/max/PARADIM-1_{}_max.npy",
    index=(1, 801)
)

interactive(children=(Text(value='/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/PARADIM-1_46e695e0-1638-4…

<function __main__.show_npy_array(template: str, index: int, **kwargs)>

In [12]:
C.light = np.load("/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/PARADIM-1_46e695e0-1638-4045-a1ff-1506854d3c13/max/PARADIM-1_801_max.npy")
C.dark = np.load("/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/PARADIM-1_46e695e0-1638-4045-a1ff-1506854d3c13/min/PARADIM-1_801_min.npy")

## Subtract the dark from the light

In [13]:
C.light -= C.dark

## Locate the peaks and create windows

In [8]:
# tp.locate?

In [19]:
C.calc_peaks_from_light_frame(25, separation=10, threshold=60)

In [20]:
C.calc_windows_from_peaks(num=100, width=31)
C.show_windows(size=10, vmax=6000)
plt.show()

## Run throught the images to calculate the intensity in the windows

In [21]:
C.calc_intensity_in_windows()

In [22]:
C.reshape_intensity()
C.calc_coords()
C.assign_q_values()

In [23]:
facet = C.show_intensity(col_wrap=8)
axes = facet.axes.flatten()
for i in range(C.windows.shape[0]):
    d = 2 * np.pi / C.windows["Q"]
    axes[i].set_title("d = {:.3f} nm".format(d.iloc[i]))
plt.show()

## Save the result

In [24]:
BASE = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/"
FILENAME = pathlib.Path(BASE).joinpath("{}_{}.nc".format(C.metadata["sample_name"], C.metadata["uid"]))
C.to_dataset().to_netcdf(FILENAME)

## Load the result again

In [10]:
BASE = "/nsls2/data/xpd/legacy/processed/xpdUser/tiff_base/"
FILENAME = pathlib.Path(BASE).joinpath("{}_{}.nc".format(C.metadata["sample_name"], C.metadata["uid"]))
ds = xr.load_dataset(FILENAME)

In [15]:
ds["intensity"].plot(col="grain", col_wrap=4)